# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
def line_remove_and_strip(df,x):
    d = list(df[x])
    y = ''
    for j in d:
         if j.endswith(y):
            r = d.index(j)
            d[r] = j.rstrip(y)
    return d
def line_remove(df,x):
    d = list(df[x])
    for i in d:
        s = i.split()
        if s[-1] in ['Area','City','D.C.']:
            s.pop()
        m = " ".join(s)
        n = re.sub("[\(\[].*?[\)\]]", "", m)
        j = re.sub(r'[^\w]', ' ', n)
        r = d.index(i)
        d[r] = j.strip()
    return d
def line_remove1(df,x):
    d = list(df[x])
    for i in d:
        j = re.findall('[A-Z][^A-Z]*', i)
        r = d.index(i)
        d[r] = " ".join(j)
    return d
def line_strip(df,x):
    d = list(df[x])
    y = '*'
    for j in d:
        if j.endswith(y):
            r = d.index(j)
            d[r] = j.rstrip(y)
    return d

In [3]:


nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
nhl_df['team'] = line_strip(nhl_df,'team')
for i in ['Atlantic Division','Metropolitan Division','Central Division','Pacific Division']:
    a = nhl_df['team'][nhl_df.team == i].index.tolist()
    for j in a:
        nhl_df.drop(j,axis=0,inplace=True)
m = ['NFL','MLB','NBA','NHL']
for i in m:
    cities[i] = line_remove(cities,i)

nhl_df = nhl_df[nhl_df['year'] == 2018]
def nhl_correlation(): 
    p = {}
    m = nhl_df.team.tolist()
    for i in m:
        k = i.split()
        for j in cities['NHL']:
            n = j.split()
            o = k[-1]
            if j.count(o) ==1:
                a = cities['NHL'][cities.NHL == j].index.tolist()
                p[i]=cities['Metropolitan area'][a[0]]
            else:
                continue
    nhl_df['team_ville'] = nhl_df['team']
    nhl_df['team_ville'] = nhl_df['team_ville'].map(p)
   

    df = (pd.merge(nhl_df,cities, left_on= "team_ville", right_on= "Metropolitan area")).head(31)

    df['W'] = pd.to_numeric(df['W'])
    df['L'] = pd.to_numeric(df['L'])
    df['Population (2016 est.)[8]'] = pd.to_numeric(df['Population (2016 est.)[8]'])
    he = ['team','W','L','Metropolitan area','Population (2016 est.)[8]']

    df = df[he]

    df['W/L'] = df['W']/(df['L']+df['W'])

    df = df.groupby('Metropolitan area').mean().reset_index()
    
    population_by_region = df['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = df['W/L'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    
    population_by_region = df['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = df['W/L'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    p,v = stats.pearsonr(population_by_region, win_loss_by_region)
    return p
nhl_correlation()

0.012486162921209907

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]] 
nba_df['team'] = line_remove(nba_df,'team')


nba_df = nba_df[nba_df['year'] == 2018]
def nba_correlation(): 
    p = {}
    m = nba_df.team.tolist()
    for i in m:
        k = i.split()
        for j in cities['NBA']:
            n = j.split()
            o = k[-1]
            if j.count(o) ==1:
                a = cities['NBA'][cities.NBA == j].index.tolist()
                p[i]=cities['Metropolitan area'][a[0]]
            else:
                continue
    print(p)
    nba_df['team_ville'] = nba_df['team']
    nba_df['team_ville'] = nba_df['team_ville'].map(p)
    nba_df.sort_values('team_ville')
    

    df = pd.merge(nba_df,cities,left_on= "team_ville", right_on= "Metropolitan area")
    df['W/L%'] = pd.to_numeric(df['W/L%'])
    df['Population (2016 est.)[8]'] = pd.to_numeric(df['Population (2016 est.)[8]'])
    he = ['team','W/L%','Metropolitan area','Population (2016 est.)[8]']

    df = df[he]

   # raise NotImplementedError()
    df = df.groupby('Metropolitan area').mean().reset_index()
    
    population_by_region = df['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = df['W/L%'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    
    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
nba_correlation()

{'Toronto Raptors': 'Toronto', 'Boston Celtics': 'Boston', 'Philadelphia 76ers': 'Philadelphia', 'Cleveland Cavaliers': 'Cleveland', 'Indiana Pacers': 'Indianapolis', 'Miami Heat': 'Miami–Fort Lauderdale', 'Milwaukee Bucks': 'Milwaukee', 'Washington Wizards': 'Washington, D.C.', 'Detroit Pistons': 'Detroit', 'Charlotte Hornets': 'Charlotte', 'New York Knicks': 'New York City', 'Brooklyn Nets': 'New York City', 'Chicago Bulls': 'Chicago', 'Orlando Magic': 'Orlando', 'Atlanta Hawks': 'Atlanta', 'Houston Rockets': 'Houston', 'Golden State Warriors': 'San Francisco Bay Area', 'Portland Trail Blazers': 'Portland', 'Oklahoma City Thunder': 'Oklahoma City', 'Utah Jazz': 'Salt Lake City', 'New Orleans Pelicans': 'New Orleans', 'San Antonio Spurs': 'San Antonio', 'Minnesota Timberwolves': 'Minneapolis–Saint Paul', 'Denver Nuggets': 'Denver', 'Los Angeles Clippers': 'Los Angeles', 'Los Angeles Lakers': 'Los Angeles', 'Sacramento Kings': 'Sacramento', 'Dallas Mavericks': 'Dallas–Fort Worth', 'Mem

-0.17636350642182935

In [5]:
nba_df[nba_df['year'] == 2018]

,team,W,L,W/L%,GB,PS/G,PA/G,SRS,year,League,team_ville
0,Toronto Raptors,59,23,0.72,—,111.7,103.9,7.29,2018,NBA,Toronto
1,Boston Celtics,55,27,0.6709999999999999,4.0,104.0,100.4,3.23,2018,NBA,Boston
2,Philadelphia 76ers,52,30,0.634,7.0,109.8,105.3,4.3,2018,NBA,Philadelphia
3,Cleveland Cavaliers,50,32,0.61,9.0,110.9,109.9,0.59,2018,NBA,Cleveland
4,Indiana Pacers,48,34,0.585,11.0,105.6,104.2,1.18,2018,NBA,Indianapolis
5,Miami Heat,44,38,0.537,15.0,103.4,102.9,0.15,2018,NBA,Miami–Fort Lauderdale
6,Milwaukee Bucks,44,38,0.537,15.0,106.5,106.8,-0.45,2018,NBA,Milwaukee
7,Washington Wizards,43,39,0.524,16.0,106.6,106.0,0.53,2018,NBA,"Washington, D.C."
8,Detroit Pistons,39,43,0.47600000000000003,20.0,103.8,103.9,-0.26,2018,NBA,Detroit
9,Charlotte Hornets,36,46,0.439,23.0,108.2,108.0,0.07,2018,NBA,Charlotte


In [6]:
cities

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils[note 3]
1,Los Angeles,13310447,RamsChargers[note 4],DodgersAngels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,49ersRaiders[note 6],GiantsAthletics,Warriors,Sharks[note 7]
3,Chicago,9512999,Bears[note 8],CubsWhite Sox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars
5,"Washington, D.C.",6131977,Redskins,Nationals[note 10],Wizards[note 11],Capitals
6,Philadelphia,6070500,Eagles,Phillies[note 12],76ers,Flyers[note 13]
7,Boston,4794447,Patriots[note 14],Red Sox[note 15],Celtics,Bruins
8,Minneapolis–Saint Paul,3551036,Vikings,Twins,Timberwolves,Wild[note 16]
9,Denver,2853077,Broncos,Rockies,Nuggets[note 17],Avalanche[note 18]


## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [30]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
mlb_df = mlb_df[mlb_df['year'] == 2018]
m = ['NFL','MLB','NBA','NHL']
for i in m:
    cities[i] = line_remove(cities,i)
def mlb_correlation(): 
    import difflib
    p = {}
    m = mlb_df.team.tolist()
    e = []
    for i in m:
        k = i.split()
        for j in cities['MLB']:
                n = j.split()
                o = k[-1]
                if j.count(o) ==1:
                    a = cities['MLB'][cities.MLB == j].index.tolist()
                    p[i]=cities['Metropolitan area'][a[0]]
    p.pop('Chicago White Sox')
    p['Chicago White Sox'] = 'Chicago'
    mlb_df['team_ville'] = mlb_df['team']
    mlb_df['team_ville'] = mlb_df['team_ville'].map(p)
    mlb_df.sort_values('team_ville')
    #raise NotImplementedError()
    df = pd.merge(mlb_df,cities,left_on= "team_ville", right_on= "Metropolitan area")
    df['W'] = pd.to_numeric(df['W'])
    df['L'] = pd.to_numeric(df['L'])
    df['Population (2016 est.)[8]'] = pd.to_numeric(df['Population (2016 est.)[8]'])
    he = ['team','W','L','Metropolitan area','Population (2016 est.)[8]']

    df = df[he]

    df['W/L'] = df['W']/(df['L']+df['W'])

    df = df.groupby('Metropolitan area').mean().reset_index()
    
    
    population_by_region = df['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = df['W/L'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"
    t,v = stats.pearsonr(population_by_region, win_loss_by_region)
    return t
mlb_correlation()

0.15027698302669307

In [18]:
mlb_df

,team,W,L,W-L%,GB,year,League,team_ville
0,Boston Red Sox,108,54,0.667,--,2018,MLB,Boston
1,New York Yankees,100,62,0.617,8.0,2018,MLB,New York City
2,Tampa Bay Rays,90,72,0.556,18.0,2018,MLB,Tampa Bay Area
3,Toronto Blue Jays,73,89,0.451,35.0,2018,MLB,Toronto
4,Baltimore Orioles,47,115,0.290,61.0,2018,MLB,Baltimore
5,Cleveland Indians,91,71,0.562,--,2018,MLB,Cleveland
6,Minnesota Twins,78,84,0.481,13.0,2018,MLB,Minneapolis–Saint Paul
7,Detroit Tigers,64,98,0.395,27.0,2018,MLB,Detroit
8,Chicago White Sox,62,100,0.383,29.0,2018,MLB,Chicago
9,Kansas City Royals,58,104,0.358,33.0,2018,MLB,Kansas City


In [89]:
cities

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets,YankeesMets,KnicksNets,RangersIslandersDevils
1,Los Angeles,13310447,RamsChargers,DodgersAngels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,49ersRaiders,GiantsAthletics,Warriors,Sharks
3,Chicago,9512999,Bears,CubsWhite Sox,Bulls,Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars
5,"Washington, D.C.",6131977,Redskins,Nationals,Wizards,Capitals
6,Philadelphia,6070500,Eagles,Phillies,76ers,Flyers
7,Boston,4794447,Patriots,Red Sox,Celtics,Bruins
8,Minneapolis–Saint Paul,3551036,Vikings,Twins,Timberwolves,Wild
9,Denver,2853077,Broncos,Rockies,Nuggets,Avalanche


## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [16]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
nfl_df = nfl_df[nfl_df['year'] == 2018]
for i in ['AFC East','AFC North','AFC South','AFC West','NFC East','NFC North', 'NFC South','NFC West']:
    a = nfl_df['team'][nfl_df.team == i].index.tolist()
    for j in a:
        nfl_df.drop(j,axis=0,inplace=True)
m = ['NFL','MLB','NBA','NHL']
for i in m:
    cities[i] = line_remove(cities,i)
nfl_df['team'] = line_remove(nfl_df,'team')
cities['NFL'] = line_remove1(cities,'NFL')
def nfl_correlation(): 
    p = {}
    m = nfl_df.team.tolist()
    for i in m:
        k = i.split()
        for j in cities['NFL']:
            n = j.split()
            o = k[-1]
            if j.count(o) ==1:
                a = cities['NFL'][cities.NFL == j].index.tolist()
                p[i]=cities['Metropolitan area'][a[0]]
            else:
                continue
    nfl_df['team_ville'] = nfl_df['team']
    nfl_df['team_ville'] = nfl_df['team_ville'].map(p)
    df = pd.merge(nfl_df,cities,left_on= "team_ville", right_on= "Metropolitan area")
    #raise NotImplementedError()
    
    df['W-L%'] = pd.to_numeric(df['W-L%'])
    df['Population (2016 est.)[8]'] = pd.to_numeric(df['Population (2016 est.)[8]'])
    he = ['team','W-L%','Metropolitan area','Population (2016 est.)[8]']

    df = df[he]

   # raise NotImplementedError()
    df = df.groupby('Metropolitan area').mean().reset_index()
    
    population_by_region = df['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = df['W-L%'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"
    p,v = stats.pearsonr(population_by_region, win_loss_by_region)
    return p
nfl_correlation()

(0.004282141436392957, 0.9824114740736537)

In [11]:
cities

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,Giants Jets,YankeesMets,KnicksNets,RangersIslandersDevils
1,Los Angeles,13310447,Rams Chargers,DodgersAngels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,Raiders,GiantsAthletics,Warriors,Sharks
3,Chicago,9512999,Bears,CubsWhite Sox,Bulls,Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars
5,"Washington, D.C.",6131977,Redskins,Nationals,Wizards,Capitals
6,Philadelphia,6070500,Eagles,Phillies,76ers,Flyers
7,Boston,4794447,Patriots,Red Sox,Celtics,Bruins
8,Minneapolis–Saint Paul,3551036,Vikings,Twins,Timberwolves,Wild
9,Denver,2853077,Broncos,Rockies,Nuggets,Avalanche


In [12]:
nfl_df

,DSRS,L,League,MoV,OSRS,PA,PD,PF,SRS,SoS,T,W,W-L%,team,year,team_ville
1,2.1,5,NFL,6.9,3.1,325,111,436,5.2,-1.8,0,11,.688,New England Patriots,2018,Boston
2,-5.2,9,NFL,-7.1,-3.6,433,-114,319,-8.8,-1.7,0,7,.438,Miami Dolphins,2018,Miami–Fort Lauderdale
3,-0.6,10,NFL,-6.6,-6.3,374,-105,269,-6.9,-0.3,0,6,.375,Buffalo Bills,2018,Buffalo
4,-5.9,12,NFL,-6.8,-2.0,441,-108,333,-7.8,-1.1,0,4,.250,New York Jets,2018,New York City
6,6.4,6,NFL,6.4,0.6,287,102,389,7.0,0.6,0,10,.625,Baltimore Ravens,2018,Baltimore
7,1.7,6,NFL,4.3,3.9,360,68,428,5.6,1.3,1,9,.594,Pittsburgh Steelers,2018,Pittsburgh
8,0.6,8,NFL,-2.1,-1.0,392,-33,359,-0.3,1.7,1,7,.469,Cleveland Browns,2018,Cleveland
9,-3.4,10,NFL,-5.4,0.0,455,-87,368,-3.4,2.0,0,6,.375,Cincinnati Bengals,2018,Cincinnati
11,1.4,5,NFL,5.4,2.4,316,86,402,3.8,-1.5,0,11,.688,Houston Texans,2018,Houston
12,-0.6,6,NFL,5.6,3.9,344,89,433,3.4,-2.2,0,10,.625,Indianapolis Colts,2018,Indianapolis


## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values